In [ ]:
# External libraries
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.style as style
import pandas as pd
# Own Libraries
from utility.utils import *
from Match.pair import *
from Match.icp import *

%matplotlib tk
style.use('seaborn-paper')

# CATALOG CREATION

In [ ]:
df = pd.read_csv('DATA/lunar_crater_database_robbins_2018.csv')
df = df[['LAT_CIRC_IMG','LON_CIRC_IMG','DIAM_CIRC_IMG']]
df.rename( columns= {'LAT_CIRC_IMG':'lat', 'LON_CIRC_IMG':'lon','DIAM_CIRC_IMG':'r'}, inplace=True )
df.r = df.r/2
df.lon = np.where(df.lon < 180, df.lon, df.lon - 360)
df = df.sort_values(by='lat')
df.insert(3,'i',range(len(df)))
df = df.sort_values(by='lon')
df.insert(4,'j',range(len(df)))
df = df[df.r < 100]
df.reset_index(drop=True, inplace=True)

In [ ]:
# init H
H = pd.DataFrame({
'Lat_min':[0],
'Lat_max':[0],
'Lon_min':[0],
'Lon_max':[0],
'ICD_max':[0],
'angle1':[0],
'angle2':[0],
'angle3':[0],
'crt_name1':[0],
'crt_name2':[0],
'crt_name3':[0]
})

In [ ]:
df.shape[0]

In [ ]:
for ind in range(400000,500000):
    triplet = find_triplet(df, ind)
    K_v = compute_K_vet(triplet)
    ICD_max = find_max_crt_dist(triplet)
    lat_m, lat_M, lon_m, lon_M = find_bounds(triplet)
    R_M = find_max_radius(triplet)
    entry = [lat_m, lat_M, lon_m, lon_M, ICD_max, K_v[0], K_v[1], K_v[2], triplet[0].name,triplet[1].name,triplet[2].name]
    if np.all(K_v > 15):
        H.loc[ind] = entry
    printProgressBar(ind, df.shape[0], printEnd='')    
    file_name = 'TRN_catalog_part_B.csv'
    H.to_csv(file_name)
    print(ind)

# Matching: 

In [1]:
# External libraries
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.style as style
import pandas as pd
import astropy
import scipy
from filterpy.kalman import KalmanFilter 
from filterpy.common import Q_discrete_white_noise
from scipy.linalg import block_diag
from astropy import units as u
from poliastro.bodies import Earth, Mars, Sun, Moon
from poliastro.twobody import Orbit
from poliastro.plotting import OrbitPlotter2D
from poliastro.plotting import OrbitPlotter3D
import glob
# Own Libraries
from utility.utils import *
from KalmanFilter.kf import *
from Detect.detector import *
from Match.pair import *
from Match.icp import *

%matplotlib tk
style.use('seaborn-paper')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [2]:
# LOAD ALL IMAGES:
dt = 5
dict = {}
for img in glob.glob(f"DATA/ephemeris sat/inclination zero/{dt} step size/*"):
    txt = img             # stringa
    t = txt.split('_')[1] # numero
    dict[t] = txt

In [3]:
# IMG1:
idx = 0
img1=cv2.imread(dict[str(idx+1)])
craters_det1 = detect(img1)
# IMG2:
idx = 1
img2=cv2.imread(dict[str(idx+1)])
craters_det2 = detect(img2)

In [4]:
df_craters_det1 = sort_mat(craters_det1)
df_craters_det2 = sort_mat(craters_det2)

In [25]:
for i in range(craters_det1.shape[0]):
    triplet = find_triplet(df_craters_det1, idx = i)
    K_v = compute_K_vet(triplet)
    if i == 0:
        QUERY1 = K_v
        TRIPLETS1 = triplet
    else:
        if np.all(K_v > 0):
            QUERY1 = np.vstack([QUERY1, K_v])
            TRIPLETS1 = np.vstack([TRIPLETS1, triplet])


for i in range(craters_det2.shape[0]):
    triplet = find_triplet(df_craters_det2, idx = i)
    K_v = compute_K_vet(triplet)
    if i == 0:
        QUERY2 = K_v
        TRIPLETS2 = triplet
    else:
        if np.all(K_v > 0):
            QUERY2 = np.vstack([QUERY2,K_v])
            TRIPLETS2 = np.vstack([TRIPLETS2,triplet])



if np.all(QUERY1[0] > 0): pass
else: 
    QUERY1 = QUERY1[1:,:]
    TRIPLETS1 = TRIPLETS1[1:,:] 

if np.all(QUERY2[0] > 0): pass
else: 
    TRIPLETS2 = TRIPLETS2[1:,:] 
    QUERY2 = QUERY2[1:,:] 

In [37]:
triplet = find_triplet(df_craters_det1, idx = 1)
K_v = compute_K_vet(triplet)
# print(triplet, K_v)
triplet

[x    794.5
 y    761.5
 r     26.5
 i     29.0
 j     27.0
 Name: 1, dtype: float64,
 x    750.00
 y    714.50
 r     12.75
 i     28.00
 j     24.00
 Name: 21, dtype: float64,
 x    750.00
 y    714.50
 r     12.75
 i     28.00
 j     24.00
 Name: 21, dtype: float64]

In [17]:
for i in QUERY1:
    for j in QUERY2:
        d = np.linalg.norm(i-j)
        if d < 1:
            print(i,j)

[  8.78330319 152.42163421  18.7950626 ] [  8.17444199 152.48999945  19.33555857]
[  8.78330319  18.7950626  152.42163421] [  8.17444199  19.33555857 152.48999945]


In [9]:
plt.figure(1)
img = img_plus_crts(img1, craters_det1)
plt.imshow(img)

plt.figure(2)
img = img_plus_crts(img2, craters_det2)
plt.imshow(img)
plt.show()

In [119]:
from numba import njit

@njit
def findAngles(a, b, c):
        # applied cosine rule
        A = np.arccos((b * b + c * c - a * a) / (2 * b * c))
        B = np.arccos((a * a + c * c - b * b) / (2 * a * c))
        C = np.arccos((b * b + a * a - c * c) / (2 * b * a))
        # convert into degrees
        A, B, C = np.rad2deg(A), np.rad2deg(B), np.rad2deg(C)
        return A, B, C


@njit
def compute_K_vet(triplet):
        a, b, c = compute_sides(triplet)
        A, B, C = findAngles(a, b, c)
        K_vet = np.array([A, B, C])
        if K_vet is not None:
            return K_vet

@njit
def compute_sides(triplet):
        a = np.linalg.norm(triplet[0][0:2] - triplet[1][0:2])
        b = np.linalg.norm(triplet[1][0:2] - triplet[2][0:2])
        c = np.linalg.norm(triplet[2][0:2] - triplet[0][0:2])
        return a, b, c


def find_all_triplets(craters):

    def Hstack(K_v, i,j,k):
        A = np.zeros(6)
        A[0],A[1],A[2] = K_v[0],K_v[1],K_v[2]
        A[3],A[4],A[5] = i,j,k
        return A
        

    def concat(a,b,c):
        A = np.zeros((3,3))
        A[0] = a
        A[1] = b
        A[2] = c
        return A

    # Input: np.array craters
    # Output: all triplets
    N = craters.shape[0]
    ender = N*N*N
    K = np.zeros((ender,6))
    lister = 0
    for i in range(N):
        for j in range(N):
            for k in range(N):
                if (i!=j) & (j!=k):
                    a =craters[i]
                    b =craters[j]
                    c =craters[k]
                    triplet = concat(a,b,c)
                    try:
                        K_v = compute_K_vet(triplet)
                        K[lister] = Hstack(K_v, i,j,k)
                    except ZeroDivisionError: pass
                    
                lister+=1
    return K[ np.all(K !=0, axis=1) ]

In [124]:
K1 = find_all_triplets(craters_det1)
K2 = find_all_triplets(craters_det2)

In [128]:
# K = pd.DataFrame(K, columns=['angle1','angle2','angle3','i','j','k'])
c = 0
for i in range(K1.shape[0]):
    for j in range(K2.shape[0]):
        a,b = K1[i], K2[j]
        d = np.linalg.norm(a-b)
        if d < 1:
            print(d)
        printProgressBar(c, K1.shape[0]*K2.shape[0], printEnd='')
        c+=1

SyntaxError: invalid syntax (<ipython-input-128-ab3a230e1898>, line 4)

In [129]:
K1.shape[0]*K2.shape[0]

327523680

In [10]:
# def matching(crat_det, crat_cat):
    
#     mat1 = crat_det
#     mat2 = crat_cat

#     MATCHER = []
#     for i in range(mat1.shape[0]):
#         for j in range(mat2.shape[0]):
#             f1 = mat1[i, :]
#             f2 = mat2[j, :]
#             tmp = sq_dif(f1,f2)
#             TMP = [i, j, tmp]
#             MATCHER.append(TMP)
#     flag = np.zeros(2)
#     for elem in MATCHER:
#         if elem[2] < 100:
#             match = [elem[0],elem[1]] 
#             flag = np.vstack((flag, match))
#     flag = flag[1:,:]
#     flag = np.array(flag).astype(int)
#     return flag


In [11]:
# flag = np.zeros(2)
# for elem in MATCHER:
#     if elem[2] < 100:
#         match = [elem[0],elem[1]] 
#         flag = np.vstack((flag, match))
# flag = flag[1:,:]
# flag = np.array(flag).astype(int)
# flag

In [12]:
# # Stack the images
# img1 = cv2.imread('/home/sirbastiano/Desktop/Python Projects/Progetto Tesi/Dataset/2 step size/IMG_1_Lat_0_Long_-147.8129.jpg')
# img2 = cv2.imread('/home/sirbastiano/Desktop/Python Projects/Progetto Tesi/Dataset/2 step size/IMG_2_Lat_0_Long_-147.7069.jpg')
# img_matched = np.hstack((img1,img2)).astype(int)
# plt.imshow(img_matched, cmap='gray')
# plt.axis(False)
# plt.grid(False)

In [13]:
# cv2.arrowedLine(img_matched,start,end,(255, 0, 0) , 3)

In [14]:
# center_coordinates = start
# radius = 500
# color = (0,0,255)
# thickness = 6
# cv2.circle(img_matched, center_coordinates, radius, color, thickness)